# Apprentice Chef - Regression Model Development

> *by Jack Daoud, February 11th 2021*

# Setup

In [1]:
# Set up environment with libraries & data

##############################################################################
# Import libraries

import pandas as pd                     # data sceince essentials
import matplotlib.pyplot as plt         # essential graphical output
import seaborn as sns                   # enhanced visualizations
import numpy as np                      # mathematical essentials
from tqdm.notebook import tqdm          # progress bars
import time                             # time essentials

# Set Pandas print options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

##############################################################################
# Import data

# Specify file name
file_data = './_data/Apprentice_Chef_Dataset.xlsx'

# Read file into Python
chefs = pd.read_excel(io     = file_data,
                      header = 0)

##############################################################################
# Fix data

# Rename LARGEST_ORDER_SIZE to AVG_MEALS_ORDERED
chefs.rename(columns={'LARGEST_ORDER_SIZE':'AVG_MEALS_ORDERED'}, inplace=True)

# Print dimensions of data
print(f"""
Size of Original Dataset
------------------------
Observations: {chefs.shape[0]}
Features:     {chefs.shape[1]}
""")


Size of Original Dataset
------------------------
Observations: 1946
Features:     28



# Feature Engineering

In [2]:
# Log transformation on REVENUE
#chefs.rename(columns={'REVENUE':'log_REVENUE'}, inplace=True)
chefs['log_REVENUE'] = np.log10(chefs['REVENUE'])

In [3]:
# TOTAL_CANCELS Variable

chefs['TOTAL_CANCELS'] = chefs['CANCELLATIONS_BEFORE_NOON'] + chefs['CANCELLATIONS_AFTER_NOON']

In [4]:
# TOTAL_MEALS_DELIVERED Variable

chefs['TOTAL_MEALS_DELIVERED'] = chefs['TOTAL_MEALS_ORDERED'] - chefs['TOTAL_CANCELS']

In [5]:
# 7 Flag-based Features

# Basic Rule:
# At least 100 observations must be on both sides of the flag (1s and 0s)

# Calculate number of observations with 0s per variable
photos             = sum(chefs['TOTAL_PHOTOS_VIEWED'][:] == 0)       # TOTAL_PHOTOS_VIEWED
early_deliveries   = sum(chefs['EARLY_DELIVERIES'][:] == 0)          # EARLY_DELIVERIES
late_deliveries    = sum(chefs['LATE_DELIVERIES'][:] == 0)           # LATE_DELIVERIES
early_cancellation = sum(chefs['CANCELLATIONS_BEFORE_NOON'][:] == 0) # CANCELLATIONS_BEFORE_NOON
late_cancellation  = sum(chefs['CANCELLATIONS_AFTER_NOON'][:] == 0)  # CANCELLATIONS_AFTER_NOON
canceled           = sum(chefs['TOTAL_CANCELS'][:] == 0)             # TOTAL_CANCELS
subscribers        = sum(chefs['WEEKLY_PLAN'][:] == 0)               # WEEKLY_PLAN
learner            = sum(chefs['MASTER_CLASSES_ATTENDED'][:] == 0)   # MASTER_CLASSES_ATTENDED


# printing a table of the results
#print(f"""
#                 No\t\tYes
#               ---------------------
#Viewd Photos   | {photos}\t\t{len(chefs) - photos}
#Early Delivery | {early_deliveries}\t\t{len(chefs) - early_deliveries}
#Late Delivery  | {late_deliveries}\t\t{len(chefs) - late_deliveries}
#Early Cancel   | {early_cancellation}\t\t{len(chefs) - early_cancellation}
#Late Cancel    | {late_cancellation}\t\t{len(chefs) - late_cancellation}
#Canceled       | {canceled}\t\t{len(chefs) - canceled}
#Subscribers    | {subscribers}\t\t{len(chefs) - subscribers}
#Learner        | {learner}\t\t{len(chefs) - learner}
#""")

##############################################################################
# Engineer Features

# dummy variables
chefs['VIEWED_PHOTOS']      = 0
chefs['EARLY_DELIVERY']     = 0
chefs['LATE_DELIVERY']      = 0
chefs['EARLY_CANCELLATION'] = 0
chefs['LATE_CANCELLATION']  = 0
chefs['CANCELED']           = 0
chefs['SUBSCRIBER']         = 0
chefs['LEARNER']            = 0

for index, value in chefs.iterrows():

    # TOTAL_PHOTOS_VIEWED
    if chefs.loc[index, 'TOTAL_PHOTOS_VIEWED'] > 0:
        chefs.loc[index, 'VIEWED_PHOTOS'] = 1
        
        
    # EARLY_DELIVERIES
    if chefs.loc[index, 'EARLY_DELIVERIES'] > 0:
        chefs.loc[index, 'EARLY_DELIVERY'] = 1
        
        
    # LATE_DELIVERIES
    if chefs.loc[index, 'LATE_DELIVERIES'] > 0:
        chefs.loc[index, 'LATE_DELIVERY'] = 1
        
        
    # EARLY_CANCELLATION
    if chefs.loc[index, 'CANCELLATIONS_BEFORE_NOON'] > 0:
        chefs.loc[index, 'EARLY_CANCELLATION'] = 1
       
    
    # LATE_CANCELLATION
    if chefs.loc[index, 'CANCELLATIONS_AFTER_NOON'] > 0:
        chefs.loc[index, 'LATE_CANCELLATION'] = 1
        
        
    # TOTAL_CANCELS
    if chefs.loc[index, 'TOTAL_CANCELS'] > 0:
        chefs.loc[index, 'CANCELED'] = 1

        
    # WEEKLY_PLAN
    if chefs.loc[index, 'WEEKLY_PLAN'] > 0:
        chefs.loc[index, 'SUBSCRIBER'] = 1
        
        
    # MASTER_CLASSES_ATTENDED
    if chefs.loc[index, 'MASTER_CLASSES_ATTENDED'] > 0:
        chefs.loc[index, 'LEARNER'] = 1



In [6]:
# LAST_NAME Binary Variable

"""
This is is a YES/NO variable for customers with a legitimate FAMILY_NAME.
Legitimate means customer's FAMILY_NAME is non-identical to the FIRST_NAME.

- A small portion of users didn't input a family name.
- Another portion inputted their family name identical to their first name.

We will group aforementioned portions into customers with NO LAST_NAME.

This variable assumes that FIRST_NAME and FAMILY_NAME shouldn't be identical.
Some may argue that some people have identical FIRST_NAME and FAMILY_NAME,
however, this is a rare occurance. 

This is based on the reasoning that people who only input first name in a lead
generation form might not be serious and not intend to buy anything (Chase, 2021).
"""

# Count the number of times a space appears in a customers NAME
chefs['LAST_NAME'] = chefs['NAME'].str.count(' ')

# looping to group observations by domain type
chefs.loc[chefs['LAST_NAME'] > 1, 'LAST_NAME'] = 1

##############################################################################
        
# Plot Number of Customers with missing family names

# Count number of churn per month
last_names = chefs['LAST_NAME'].value_counts()

# Create plot
ax = last_names.plot(kind='bar', 
                      legend = True,
                      figsize = (8,6),
                      rot = 0,
                      colormap = "Paired")


# Annotate plot with values
for p in ax.patches:
    ax.annotate(format(p.get_height(), ',.0f'),
                (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center',
                va='center',
                xytext=(0, 6),
                textcoords='offset points')

# Set plot aesthetics
ax.set_ylabel('# Customers',size = 14)
ax.set_xlabel('')
ax.set_title('Customers with Family Names', size = 14)
ax.set_xticklabels(labels = ['Has Family Name', 'No Family Name'])
ax.legend().remove()

# Display plot
#plt.show()
plt.close()

In [7]:
# MEAL_POLARITY Variable

"""
This is a ordinal variable that classifies customer polarity based off
their MEDIAN_MEAL_RATING. 
- If the rating is greater than 3 (midpoint), then customer polarity is positive
- If the rating is less than 3, then customer polarity is negative
- If the rating equals 3, then customer polarity neutral

Positive = 1
Neutral  = 0
Negative = -1
"""

# Loop through each customer
for index, column in chefs.iterrows():
    
    # Check if the customer has a positive experience on average
    if chefs.loc[ index , 'MEDIAN_MEAL_RATING'] > 3:
        chefs.loc[ index , 'MEAL_POLARITY'] = 1
    elif chefs.loc[ index , 'MEDIAN_MEAL_RATING'] < 3:
        chefs.loc[ index , 'MEAL_POLARITY'] = -1
    else:
        chefs.loc[ index , 'MEAL_POLARITY'] = 0
        
        
# Change data type to integer
chefs.MEAL_POLARITY = chefs.MEAL_POLARITY.astype(int)

##############################################################################
# Plot Polarity by Customers

# Count number of churn per month
domains = chefs['MEAL_POLARITY'].value_counts()

# Create plot
ax = domains.plot(kind='bar', 
                  legend = True,
                  figsize = (8,6),
                  rot = 0,
                  colormap = "Paired")


# Annotate plot with values
for p in ax.patches:
    ax.annotate(format(p.get_height(), '.0f'),
                (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center',
                va='center',
                xytext=(0, 6),
                textcoords='offset points')

# Set plot aesthetics
ax.set_ylabel('# Customers',size = 14)
ax.set_xlabel('')
ax.set_title('Polartiy of Customers towards Meal Sets', size = 14)
ax.set_xticklabels(labels = ['Neutral', 'Negative', 'Positive'])
ax.legend().remove()
 
# Display plot
#plt.show()
plt.close()

In [8]:
# COMPLAINED Binary Variable

"""
This is a binary variable that classifies whether or not a customer has
complained.

The customer team claims that the vast majority of cases are complaints.
Therefore, a reasonible demarcation of whether or not a customer 
complained would be:

If the number contacts a customer has with customer service is higher than
the average (7 contacts), then that customer is assumed to have complained.
"""

# Create complained variable and set it to 0
chefs['COMPLAINED'] = 0

# Calculate average of contacts with customer_service
AVG_CONTACTS_W_CUSTOMER_SERVICE = chefs['CONTACTS_W_CUSTOMER_SERVICE'].mean()

# Loop through each customer
for index, column in chefs.iterrows():
    
    # Check if the customer has a positive experience on average
    if chefs.loc[ index , 'CONTACTS_W_CUSTOMER_SERVICE'] > AVG_CONTACTS_W_CUSTOMER_SERVICE:
        chefs.loc[ index , 'COMPLAINED'] = 1
        
##############################################################################       
# Plot Number of Complaining Customers

# Count number of customers who complained
domains = chefs['COMPLAINED'].value_counts()

# Create plot
ax = domains.plot(kind='bar', 
                  legend = True,
                  figsize = (8,6),
                  rot = 0,
                  colormap = "Paired")


# Annotate plot with values
for p in ax.patches:
    ax.annotate(format(p.get_height(), '.0f'),
                (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center',
                va='center',
                xytext=(0, 6),
                textcoords='offset points')

# Set plot aesthetics
ax.set_ylabel('# Customers',size = 14)
ax.set_xlabel('')
ax.set_title('Number of Complaining Customers', size = 14)
ax.set_xticklabels(labels = ['Complained', "Didn't Complain"])
ax.legend().remove()
 
# Display plot
#plt.show()
plt.close()

In [9]:
# DOMAIN_GROUP Categorical Variable

"""
This is a categorical variable that classifies email domains based on the 
info provided by the marketing team.
"""

# Loop through each customer
for index, col in chefs.iterrows():
    
        # Check if the customer is personal
        if '@gmail.com' in chefs.loc[index, 'EMAIL'] \
        or '@yahoo.com' in chefs.loc[index, 'EMAIL'] \
        or '@protonmail.com' in chefs.loc[index, 'EMAIL']:
            chefs.loc[index, 'DOMAIN_GROUP'] = 'PERSONAL'
        elif '@me.com' in chefs.loc[index, 'EMAIL'] \
        or '@aol.com' in chefs.loc[index, 'EMAIL'] \
        or '@hotmail.com' in chefs.loc[index, 'EMAIL'] \
        or '@live.com' in chefs.loc[index, 'EMAIL'] \
        or '@msn.com' in chefs.loc[index, 'EMAIL'] \
        or '@passport.com' in chefs.loc[index, 'EMAIL']:
            chefs.loc[index, 'DOMAIN_GROUP'] = 'JUNK'
        else:
            chefs.loc[index, 'DOMAIN_GROUP'] = 'PROFESSIONAL'

##############################################################################
# Plot Number of Customers per Domain Group

# Count number of domain groups
domains = chefs['DOMAIN_GROUP'].value_counts()

# Create plot
ax = domains.plot(kind='bar', 
                  legend = True,
                  figsize = (8,6),
                  rot = 0, 
                  colormap = "Paired")


# Annotate plot with values
for p in ax.patches:
    ax.annotate(format(p.get_height(), '.0f'),
                (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center',
                va='center',
                xytext=(0, 6),
                textcoords='offset points')

# Set plot aesthetics
ax.set_ylabel('# Customers',size = 14)
ax.set_xlabel('')
ax.set_title('Number of Customers per Domain Group', size = 14)
ax.legend().remove()

# Display plot
#plt.show()
plt.close()

In [10]:
# GENDER Variable from GoT Dataset

"""
1 denotes Male
0 denotes Female
"""


# Import Game of Thrones character data
genders = pd.read_csv('./_data/Game_of_Thrones_Characters.csv')

# Select only name & gender columns
genders = genders[['name', 'male']]

# Rename columns to match Chefs DataFrame
genders.columns = ['NAME', 'GENDER']

# Merge Gender column to Chefs DataFrame based on NAMES
chefs = pd.merge(chefs,  
                 genders,  
                 on ='NAME',  
                 how ='left') 

##############################################################################
# Plot Gender of Customers

# Count number of genders
genders = chefs['GENDER'].value_counts()

# Create plot
ax = genders.plot(kind='bar', 
                  legend = True,
                  figsize = (8,6),
                  rot = 0,
                  colormap = "Paired")


# Annotate plot with values
for p in ax.patches:
    ax.annotate(format(p.get_height(), '.0f'),
                (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center',
                va='center',
                xytext=(0, 6),
                textcoords='offset points')

# Set plot aesthetics
ax.set_ylabel('# Customers',size = 14)
ax.set_xlabel('')
ax.set_title("Customers' Gender", size = 14)
ax.set_xticklabels(labels = ['Male', "Female"])
ax.legend().remove()
 
# Display plot
#plt.show()
plt.close()

In [11]:
# Convert DOMAIN_GROUP into Dummy Variables

# Drop PII variables
chefs.drop(labels  = ['NAME', 'EMAIL', 'FIRST_NAME', 'FAMILY_NAME'],
           axis    = 1,
           inplace = True)

# Get Dummies 
chefs = pd.get_dummies(chefs)

# Drop DOMAIN_GROUP_JUNK to abide by n-1 rule for dummies
chefs.drop('DOMAIN_GROUP_JUNK', axis = 1, inplace = True)

In [12]:
# Outlier-based Features

# Define function to flag outliers
def flag_outliers(data, variable):
    """
    This function takes in a dataframe and a variable.
    
    Then, it calculates the 25th and 75th quantiles of the variable.
    
    Then, it flags any data point that precedes the 25th and exceeds the 75th
    quantiles. This flag is returned in a new variable labelled out_VARIABLE.
    
    Then, it deletes any out_VARIABLE with 0 flags.
    
    -----------
    Parameters:
    - DATA 
    - VARIABLE
    """
    # Create placeholder for outlier flag
    data['out_'+variable] = 0
    
    # Set upper and lower thresholds
    lower_threshold = pd.DataFrame(data.quantile(.25, axis = 0))
    upper_threshold = pd.DataFrame(data.quantile(.75, axis = 0))
    
    # Rename columns for indexing
    lower_threshold.columns = ['LOWER']
    upper_threshold.columns = ['UPPER']
    
    # Loop over each column to change its respective flag
    for index, column in data.iterrows():
        
        # Set conditional for variable & upper threshold
        if data.loc[index, variable] > upper_threshold.loc[variable, 'UPPER']:
            
            # Change outlier flag
            data.loc[index, 'out_'+variable] = 1
            
        # Set conditional for variable & lower threshold
        if data.loc[index, variable] < lower_threshold.loc[variable, 'LOWER']:
        
            # Change outlier flag
            data.loc[index, 'out_'+variable] = 1
    
    # Check for any variables with < 100 outliers and delete them
    if data['out_'+variable].sum() < 100:
        del data['out_'+variable]


# List of original variables
original_variables = ['CROSS_SELL_SUCCESS', 'TOTAL_MEALS_ORDERED', 
                      'UNIQUE_MEALS_PURCH', 'CONTACTS_W_CUSTOMER_SERVICE', 
                      'PRODUCT_CATEGORIES_VIEWED', 'AVG_TIME_PER_SITE_VISIT',
                      'TASTES_AND_PREFERENCES', 
                      'PC_LOGINS', 'MOBILE_LOGINS', 'WEEKLY_PLAN', 
                      'EARLY_DELIVERIES', 'LATE_DELIVERIES', 'PACKAGE_LOCKER', 
                      'AVG_PREP_VID_TIME', 'TOTAL_PHOTOS_VIEWED',
                      'AVG_MEALS_ORDERED', 'MASTER_CLASSES_ATTENDED', 
                      'MEDIAN_MEAL_RATING', 'AVG_CLICKS_PER_VISIT']

# Variables excluded from outlier flagging:
# REVENUE
# MOBILE_NUMBER
# CANCELLATIONS_BEFORE_NOON
# CANCELLATIONS_AFTER_NOON
# REFRIGERATED_LOCKER


# Run function to flag outliers for original variables
for col in chefs.columns:
    if col in original_variables:
        flag_outliers(chefs, col)
        
        
##############################################################################       
# Plot outlier flags        
def plot_outlier_flags(data, variable):
    """
    This function will take a dataframe and variable and plot the count
    of that variable in a bar plot.
    """
    # Create plot
    ax = data[variable].value_counts().plot(kind     = 'bar',
                                            figsize  = (8,6),
                                            rot      = 0,
                                            colormap = 'Paired')
    
    # Annotate plot with values
    for p in ax.patches:
        ax.annotate(format(p.get_height(), '.0f'),
                    (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha='center',
                    va='center',
                    xytext=(0, 6),
                    textcoords='offset points')
        
    # Set plot aesthetics
    ax.set_ylabel('# Customers',size = 14)
    ax.set_xlabel('')
    ax.set_title(variable, size = 14)
    
    # Display plot
    plt.show()

    
# Loop through outlier flags and plot bar plots
#for col in chefs.columns:
#    if 'out_' in col:
#        plot_outlier_flags(chefs, col)

In [13]:
# K-Means Clustering - Customer Segment Variable
from sklearn.cluster import KMeans                 # Customer segmentation

# Subset for explanatory variables for customer segmentation
X = chefs.loc[:, :].values

# Instantiate number of clusters
clusterNum = 3

# Instantiate KMEANS model
k_means = KMeans(init         = "k-means++", 
                 n_clusters   = clusterNum, 
                 n_init       = 12,
                 random_state = 219)

# Fit the model
k_means.fit(X)

# Grab each label (segmentation) for each customer
k_means_labels = k_means.labels_

# Add cluster labels to data set
chefs["KMEANS_CLUSTERS"] = k_means_labels

# Display cluster average per variable
#chefs.groupby('KMEANS_CLUSTERS').mean()

##############################################################################
# Plot 

# Count number of churn per month
genders = chefs['KMEANS_CLUSTERS'].value_counts()

# Create plot
ax = genders.plot(kind='bar', 
                  legend = True,
                  figsize = (8,6),
                  rot = 0,
                  colormap = "Paired")


# Annotate plot with values
for p in ax.patches:
    ax.annotate(format(p.get_height(), '.0f'),
                (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center',
                va='center',
                xytext=(0, 6),
                textcoords='offset points')

# Set plot aesthetics
ax.set_ylabel('# Customers',size = 14)
ax.set_xlabel('')
ax.set_title("Customer Segments", size = 14)
#ax.set_xticklabels(labels = ['Segment 01', "Segment 02", "Segment 03"])
ax.legend().remove()
 
# Display plot
#plt.show()
plt.close()


##############################################################################
# Plot elbow point for optimal K

# Placeholder
#distortions = []

# Number of Ks
#K = range(1,10)

# Loop through each K
#for k in K:
#    kmeanModel = KMeans(n_clusters=k)
#    kmeanModel.fit(X)                            # Fit with X
#    distortions.append(kmeanModel.inertia_)      # Append inertia to distortions
    
# "Inertia can be recognized as a measure of how internally coherent clusters are."
# https://scikit-learn.org/stable/modules/clustering.html

# Plot elbow point
#plt.figure(figsize=(16,8))
#plt.plot(K, distortions, 'bx-')
#plt.xlabel('k')
#plt.ylabel('Distortion')
#plt.title('The Elbow Method showing the optimal k')
#plt.show()

# Source:
# https://predictivehacks.com/k-means-elbow-method-code-for-python/

In [14]:
# Convert KMEANS_CLUSTERS into Dummy Variables

# Get dummy variables for KMEANS_CLUSTERS
one_hot_Clusters = pd.get_dummies(chefs['KMEANS_CLUSTERS'])

# Rename columns
one_hot_Clusters.columns = ['KMEANS_CLUSTER_0', 'KMEANS_CLUSTER_1', 'KMEANS_CLUSTER_2']

# Drop cluster 0 to follow n-1 
one_hot_Clusters.drop('KMEANS_CLUSTER_0', axis = 1, inplace = True)


# Drop KMEANS_CLUSTER 
chefs = chefs.drop('KMEANS_CLUSTERS', axis = 1)

# Join dummy variables back to Chefs dataframe
chefs = chefs.join([one_hot_Clusters])

In [15]:
# Export feature rich dataset to excel
chefs.to_excel(excel_writer = './_data/Feature_Rich_Dataset.xlsx',
                            index = False)

# Size of feature rich data
print(f"""
Size of Feature Rich Dataset
-----------------------------
Observations: {chefs.shape[0]}
Features:     {chefs.shape[1]}
""")


Size of Feature Rich Dataset
-----------------------------
Observations: 1946
Features:     57



# Regression Model

In [16]:
# Run OLS model & print summary

# Libraries
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from regressors import stats


# Instantiate independant variables
X = chefs.loc[:, chefs.columns[
    ~chefs.columns.isin([
        'REVENUE','log_REVENUE',        # y-variables
        'CANCELLATIONS_BEFORE_NOON',    # multi-collinearity
        'TOTAL_MEALS_ORDERED',          #
        'MEDIAN_MEAL_RATING',           #
        'CANCELLATIONS_AFTER_NOON',     #
        'MASTER_CLASSES_ATTENDED',      #
        'EARLY_CANCELLATION',           #
        'EARLY_DELIVERIES',             #
        'out_TOTAL_PHOTOS_VIEWED',      #
        'AVG_MEALS_ORDERED',            #
        'TASTES_AND_PREFERENCES',       # p-value > 0.5
        'LATE_DELIVERY',                #
        'DOMAIN_GROUP_PERSONAL',        #
        'out_AVG_PREP_VID_TIME',        #
        'TOTAL_CANCELS',                # 
        'SUBSCRIBER',                   #
        'LATE_DELIVERIES',              #
        'GENDER',                       #
        'out_PRODUCT_CATEGORIES_VIEWED',#
        'PRODUCT_CATEGORIES_VIEWED',    #
        'out_LATE_DELIVERIES',          #
        'EARLY_DELIVERY',               #
        'PC_LOGINS',                    #
        'out_EARLY_DELIVERIES',         #
        'VIEWED_PHOTOS',                #
        'out_AVG_CLICKS_PER_VISIT',     #
        'TOTAL_PHOTOS_VIEWED',          #
        'DOMAIN_GROUP_PROFESSIONAL',    #
        'CANCELED',                     #
        'out_MASTER_CLASSES_ATTENDED',  #
        'CROSS_SELL_SUCCESS',           #
        'out_AVG_TIME_PER_SITE_VISIT',  #
        'REFRIGERATED_LOCKER',          #
        'AVG_CLICKS_PER_VISIT',         #
        'out_MEDIAN_MEAL_RATING',       #
        'LAST_NAME',                    #
        'LATE_CANCELLATION'])]]

# Instantiate dependant variable
y = chefs.loc[:, 'log_REVENUE']


# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.25, random_state = 219)


# Instantiate linear regression class
linear_regressor = LinearRegression()

# Fit the model
linear_regressor.fit(X_train, y_train)

# Predict log_revenue
pred_y = linear_regressor.predict(X_test)

# Create a dataframe of variable coefficients
coeff = pd.DataFrame(linear_regressor.coef_, X.columns, columns=['Coefficient'])
#print(coeff)

# Create a dataframe of actual vs predicted log_revenue
actual_vs_predicted = pd.DataFrame({'Actual': y_test, 'Predicted': pred_y})
#print(actual_vs_predicted)

# Scores
linear_train_score = linear_regressor.score(X_train, y_train).round(4) # using R-square
linear_test_score  = linear_regressor.score(X_test, y_test).round(4)   # using R-square
linear_test_gap    = abs(linear_train_score - linear_test_score).round(4)

##############################################################################
# Print model summary

# Instantiate list of used explarotary variable names
xlabels = chefs.columns[
    ~chefs.columns.isin([
        'REVENUE','log_REVENUE',        # y-variables
        'CANCELLATIONS_BEFORE_NOON',    # multi-collinearity
        'TOTAL_MEALS_ORDERED',          #
        'MEDIAN_MEAL_RATING',           #
        'CANCELLATIONS_AFTER_NOON',     #
        'MASTER_CLASSES_ATTENDED',      #
        'EARLY_CANCELLATION',           #
        'EARLY_DELIVERIES',             #
        'out_TOTAL_PHOTOS_VIEWED',      #
        'AVG_MEALS_ORDERED',            #
        'TASTES_AND_PREFERENCES',       # p-value > 0.5
        'LATE_DELIVERY',                #
        'DOMAIN_GROUP_PERSONAL',        #
        'out_AVG_PREP_VID_TIME',        #
        'TOTAL_CANCELS',                # 
        'SUBSCRIBER',                   #
        'LATE_DELIVERIES',              #
        'GENDER',                       #
        'out_PRODUCT_CATEGORIES_VIEWED',#
        'PRODUCT_CATEGORIES_VIEWED',    #
        'out_LATE_DELIVERIES',          #
        'EARLY_DELIVERY',               #
        'PC_LOGINS',                    #
        'out_EARLY_DELIVERIES',         #
        'VIEWED_PHOTOS',                #
        'out_AVG_CLICKS_PER_VISIT',     #
        'TOTAL_PHOTOS_VIEWED',          #
        'DOMAIN_GROUP_PROFESSIONAL',    #
        'CANCELED',                     #
        'out_MASTER_CLASSES_ATTENDED',  #
        'CROSS_SELL_SUCCESS',           #
        'out_AVG_TIME_PER_SITE_VISIT',  #
        'REFRIGERATED_LOCKER',          #
        'AVG_CLICKS_PER_VISIT',         #
        'out_MEDIAN_MEAL_RATING',       #
        'LAST_NAME',                    #
        'LATE_CANCELLATION'])]

# Print model output:
print(f"""
=================== MODEL SUMMARY ===================


Model Type: Ordinary Least Squares (OLS)

Model Size: {len(xlabels) + 1}


---------------------------------


OLS Training Score : {linear_regressor.score(X_train, y_train).round(4)}
OLS Testing Score  : {linear_regressor.score(X_test, y_test).round(4)}
OLS Train-Test Gap : {linear_test_gap}


---------------------------------


Mean Absolute Error     : {metrics.mean_absolute_error(y_test, pred_y).round(4)}
Mean Squared Error      : {metrics.mean_squared_error(y_test, pred_y).round(4)}
Root Mean Squared Error : {np.sqrt(metrics.mean_squared_error(y_test, pred_y)).round(4)}


---------------------------------

""")

stats.summary(linear_regressor, X, y, xlabels)

# Code for SKLEARN p-values comes from:
# https://stackoverflow.com/questions/44495667/calculate-p-value-in-sklearn-using-python


=================== MODEL SUMMARY ===================


Model Type: Ordinary Least Squares (OLS)

Model Size: 20


---------------------------------


OLS Training Score : 0.8695
OLS Testing Score  : 0.857
OLS Train-Test Gap : 0.0125


---------------------------------


Mean Absolute Error     : 0.0599
Mean Squared Error      : 0.0069
Root Mean Squared Error : 0.0828


---------------------------------


Residuals:
    Min      1Q  Median      3Q    Max
-0.2467 -0.0441 -0.0028  0.0418  0.815


Coefficients:
                                 Estimate  Std. Error   t value   p value
_intercept                       3.425775    0.014290  239.7403  0.000000
UNIQUE_MEALS_PURCH              -0.006571    0.000692   -9.4999  0.000000
CONTACTS_W_CUSTOMER_SERVICE     -0.016690    0.000805  -20.7384  0.000000
AVG_TIME_PER_SITE_VISIT         -0.000075    0.000026   -2.8548  0.004351
MOBILE_NUMBER                    0.013526    0.005185    2.6089  0.009154
MOBILE_LOGINS                   -0.008011

In [17]:
# Export predicted, actual revenue, and the deviations into excel

prediction_results = pd.DataFrame(data = {
    'Original Sale Prices' : y_test,
    'LR Predictions'       : pred_y.round(decimals = 2),
    'LR Deviation'         : pred_y.round(decimals = 2) - y_test
    })

prediction_results.to_excel(excel_writer = './_data/Linear_Model_Predictions.xlsx',
                            index = False)